In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import json

def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

path_json = '/Users/gleb/Desktop/InternetLab/internetlab/docs/clean_docs/clean_fed.json'

title = load_data(path_json)["title"]
content = load_data(path_json)["text"]

title[1]

'Концепция формирования информационного общества в России'

In [18]:
# tf-idf

vectorizer = TfidfVectorizer(
    lowercase=True,
    max_features=100,
    max_df=0.8, # рейтинг слов больше 80% будут игнорироваться
    min_df=3, # слова которые появляются реже 3 раз - игнорируются
    ngram_range = (2,4), # n-gram от одного слова до фразы из 3х
    stop_words = stopwords.words('russian')
)

vectors = vectorizer.fit_transform(content)

features_name = vectorizer.get_feature_names_out() 

dense = vectors.todense()
denselist = dense.tolist()

all_keywords = []

for desc in denselist:
    x=0
    keywords = []
    for word in desc:
        if word > 0:
            keywords.append(features_name[x])
        x +=1
    all_keywords.append(keywords)

In [21]:
with open ("/Users/gleb/Desktop/InternetLab/internetlab/docs/clean_docs/f_f_c_tf_results", "w", encoding="utf-8") as d:
    for i, j in enumerate(all_keywords):
        d.write('\n')
        d.write(f'- - - {title[i].upper()} - - - ')    
        d.write('\n')
        for w in range(len(j)):
            d.write(all_keywords[i][w])
            d.write('\n')

во всех документах упоминается безопасность гос-ва